In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import os
from itertools import combinations
from pathlib import Path
from datetime import datetime
import pickle
import json
from dataclasses import dataclass
from typing import Callable, List

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [8]:
from models.handnet_based_model import handnet_based_model
from util.training import init_device
from util.training.dataloader import split_data_for_multiple_location, concat_and_shuffle
from util.training.metrics import IntersectionOverUnion, MeanPixelAccuracy

In [9]:
PROJECT_DIRPATH = Path('/tf/workspace/deformation-prediction-multi-environment')
NAS_DIRPATH = Path('/tf/nas/')

In [10]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


# 準備
---

## デバイスの初期化

In [11]:
!nvidia-smi

Tue Nov  5 17:05:16 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:18:00.0 Off |                  Off |
| 44%   70C    P2   192W / 300W |   8354MiB / 48682MiB |     81%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:3B:00.0 Off |                  Off |
| 50%   

In [12]:
# 使用するGPUを指定
gpu = [2]

In [13]:
seed = 42
init_device(seed, gpu)

Setting random seed: 42
Random seed set for Python, NumPy, and TensorFlow.
Detected 5 GPU(s): ['/physical_device:GPU:0', '/physical_device:GPU:1', '/physical_device:GPU:2', '/physical_device:GPU:3', '/physical_device:GPU:4']
Visible GPU devices (1): ['/physical_device:GPU:2']


## ハイパーパラメータクラスの定義

In [14]:
@dataclass
class HyperParameters:
    epochs: int
    batch_size: int
    loss_function: Callable
    metrics: List[Callable]
    optimizer: Callable

## モデルの初期化関数の定義

In [15]:
def init_model(hparam):
    model = handnet_based_model(
        input_shape = (10, 52, 2),
        num_block1 = 3,
        num_block2 = 3,
        num_residual_blocks = 14, # 残差ブロックは増やすと重くなる
    )
    
    model.compile(
        optimizer = hparam.optimizer,
        loss = hparam.loss_function,
        metrics = hparam.metrics,
    )

    return model

## コールバック準備関数の定義

In [16]:
def prepare_callbacks(hparam, log_dirpath):

    callbacks = [
        TensorBoard(log_dir=log_dirpath, histogram_freq=1),
        EarlyStopping(
            monitor='val_iou',
            mode='max',
            patience=10,
            verbose=1,
            restore_best_weights=True,
        )
    ]

    return callbacks

## モデルを訓練する関数の定義

In [38]:
def save_history(save_dirpath, history):
    history_filepath = save_dirpath/'history.pkl'
    with open(history_filepath, 'wb') as file:
        pickle.dump(history, file)

def save_model(save_dirpath, model):
    # モデルのアーキテクチャを保存
    model_json = model.to_json()
    model_json_filepath = save_dirpath/'model_architecture.json'
    with open(model_json_filepath, 'w') as json_file:
        json_file.write(model_json)

    # モデルを保存
    model_filepath = save_dirpath/'model.h5'
    model.save(model_filepath)

def save_results(save_dirpath, model, test_data):
    X_test, y_test = test_data
    results = model.evaluate(X_test, y_test)
    with open(save_dirpath/'evaluation_results.txt', 'w') as f:
        for name, value in zip(model.metrics_names, results):
            f.write(f"{name}: {value:.04f}\n")
        

In [39]:
def train(hparam, train_data, valid_data, date, time, train_id):
    # データの用意
    X_train, Y_train = train_data
    X_valid, Y_valid = valid_data

    # モデルを作成
    model = init_model(hparam)

    # ディレクトリの設定
    train_dirpath = NAS_DIRPATH/'training'/date/time/train_id
    log_dirpath = PROJECT_DIRPATH/'logs'/date/time/train_id

    train_dirpath.mkdir(parents=True, exist_ok=True)

    # モデルのフィッティング
    history = model.fit(
        X_train, Y_train,
        validation_data=(X_valid, Y_valid),
        epochs = hparam.epochs,
        batch_size = hparam.batch_size,
        verbose = 1,
        callbacks = prepare_callbacks(hparam, log_dirpath)
    )
    save_history(train_dirpath, history)
    save_model(train_dirpath, model)

    return history, model

# モデルの訓練
---

In [40]:
def get_train_id(loc_list, train_loc_list):
    idx = lambda x: loc_list.index(x)

    return f"{''.join(map(str, map(idx, train_loc_list)))}"

In [41]:
def get_experiment_id(loc_list, train_loc_list, test_location):
    idx = lambda x: loc_list.index(x)

    return f"{get_train_id(loc_list, train_loc_list)}-{idx(test_location)}"

In [50]:
def main():
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H-%M-%S")
    
    # データをロード
    csi_preprocess_id = 'real_and_imag'
    location_list = ['511', '512', '514B']
    dataset_dict = split_data_for_multiple_location(csi_preprocess_id, location_list)
    train_data_dict, valid_data_dict, test_data_dict = dataset_dict

    for r in range(1, len(location_list) + 1):
        for train_location_list in combinations(location_list, r):
            print(f'train_location: {list(train_location_list)}')

            train_id = get_train_id(location_list, train_location_list)
            train_data = concat_and_shuffle(train_data_dict, train_location_list)
            valid_data = concat_and_shuffle(valid_data_dict, train_location_list)

            # ハイパーパラメータの指定
            hparam = HyperParameters(
                epochs = 150,
                batch_size = 256,
                optimizer = Adam(learning_rate=0.001),
                loss_function = BinaryFocalCrossentropy(gamma=2), # BinaryCrossentropy()
                metrics = [IntersectionOverUnion(threshold=0.5), MeanPixelAccuracy()],
            )
    
            history, model = train(hparam, train_data, valid_data, date, time, train_id)
            
            for test_location in location_list:
                experiment_id = get_experiment_id(location_list, train_location_list, test_location)
                print(f"experiment_id = {experiment_id}")
                test_data = test_data_dict[test_location]

                experiment_dirpath = NAS_DIRPATH/'results'/date/time/experiment_id
                experiment_dirpath.mkdir(parents=True, exist_ok=True)
                save_results(experiment_dirpath, model, test_data)

In [51]:
main()

train_location: ['511']
Epoch 1/150
33/33 [==============================] - 56s 261ms/step - loss: 0.1093 - iou: 0.1306 - mpa: 0.5585 - val_loss: 0.1403 - val_iou: 0.2034 - val_mpa: 0.5989
Epoch 2/150
33/33 [==============================] - 7s 206ms/step - loss: 0.0895 - iou: 0.2803 - mpa: 0.6403 - val_loss: 0.1204 - val_iou: 0.1953 - val_mpa: 0.5954
Epoch 3/150
33/33 [==============================] - 7s 209ms/step - loss: 0.0877 - iou: 0.2939 - mpa: 0.6470 - val_loss: 0.1098 - val_iou: 0.2096 - val_mpa: 0.6029
Epoch 4/150
33/33 [==============================] - 7s 217ms/step - loss: 0.0868 - iou: 0.3056 - mpa: 0.6534 - val_loss: 0.1045 - val_iou: 0.2149 - val_mpa: 0.6051
Epoch 5/150
33/33 [==============================] - 7s 207ms/step - loss: 0.0862 - iou: 0.3105 - mpa: 0.6560 - val_loss: 0.0988 - val_iou: 0.2439 - val_mpa: 0.6202
Epoch 6/150
33/33 [==============================] - 7s 208ms/step - loss: 0.0841 - iou: 0.3418 - mpa: 0.6730 - val_loss: 0.0961 - val_iou: 0.2231 - v